In [2]:
import os 

In [3]:
directory = './SentenceCorpus/labeled_articles' 

In [4]:
files = os.listdir(directory) 

In [5]:
texts = filter(lambda x: x.endswith('.txt'), files) 

In [6]:
b=list(texts)

In [7]:
b

['arxiv_annotate10_7_1.txt',
 'arxiv_annotate10_7_2.txt',
 'arxiv_annotate10_7_3.txt',
 'arxiv_annotate1_13_1.txt',
 'arxiv_annotate1_13_2.txt',
 'arxiv_annotate1_13_3.txt',
 'arxiv_annotate2_66_1.txt',
 'arxiv_annotate2_66_2.txt',
 'arxiv_annotate2_66_3.txt',
 'arxiv_annotate3_80_1.txt',
 'arxiv_annotate3_80_2.txt',
 'arxiv_annotate3_80_3.txt',
 'arxiv_annotate4_168_1.txt',
 'arxiv_annotate4_168_2.txt',
 'arxiv_annotate4_168_3.txt',
 'arxiv_annotate5_240_1.txt',
 'arxiv_annotate5_240_2.txt',
 'arxiv_annotate5_240_3.txt',
 'arxiv_annotate6_52_1.txt',
 'arxiv_annotate6_52_2.txt',
 'arxiv_annotate6_52_3.txt',
 'arxiv_annotate7_268_1.txt',
 'arxiv_annotate7_268_2.txt',
 'arxiv_annotate7_268_3.txt',
 'arxiv_annotate8_81_1.txt',
 'arxiv_annotate8_81_2.txt',
 'arxiv_annotate8_81_3.txt',
 'arxiv_annotate9_279_1.txt',
 'arxiv_annotate9_279_2.txt',
 'arxiv_annotate9_279_3.txt',
 'jdm_annotate10_210_1.txt',
 'jdm_annotate10_210_2.txt',
 'jdm_annotate10_210_3.txt',
 'jdm_annotate1_103_1.txt',
 'j

In [8]:
data_1=list()
for elem in b:
    f = open(directory+'/'+elem, 'r')
    lines = f.readlines()
    data_1.extend(lines)
    f.close()

In [9]:
def annotation(x):
    if x.find('###')==-1:
        return 1
    else:
        return 0

In [10]:
data_2 = filter(annotation, data_1)
data_2 = list(data_2)

In [11]:
import re

In [12]:
for counter, value in enumerate(data_2):
    data_2[counter]=re.sub(r'[^a-zA-Z ]','',value.replace('\n','').replace('\t','').lower())

In [13]:
ss='ownx in this paper we just scratch at the surface of lorp'

In [14]:
ss[4:-1]

' in this paper we just scratch at the surface of lor'

In [15]:
directory = './SentenceCorpus/word_lists' 
files = os.listdir(directory) 
words = filter(lambda x: x.endswith('.txt'), files) 
words=list(words)

In [16]:
words

['aim.txt', 'base.txt', 'contrast.txt', 'own.txt', 'stopwords.txt']

In [25]:
def getwords(wordlist):
    f = open(directory+'/'+wordlist, 'r')
    lines = f.readlines()
    f.close()
    for counter, value in enumerate(lines):
        lines[counter]=value.replace('\n','').lower()
    return lines

In [31]:
import Levenshtein as lv
s = np.random.seed(42)
#Мой самописный класс для создания словаря с 4-векторами для каждого слова.
#Слова близкие по Левенштейну(+опечатки) находятся ближе друг к другу в 4-пространстве
class WordList:
    def __init__(self):
        self.dict={}
    def Push(self, words, vector):
        if type(words)==list:
            for elem in words:
                self.dict.update( {elem : vector} )
        if type(words)==str:
            self.dict.update( {words : vector} )
    def GetKeys(self):
        return list(self.dict.keys())
    def GetVal(self,key):
        return self.dict.get(key)
    def FindClosest(self,word):
        closest=''
        dist=100
        for elem in self.GetKeys():
            dist_this=lv.distance(elem,word)
            if dist_this<dist :
                closest=elem
                dist=dist_this
        return {'dist':dist,'cls':closest}
    def FindVec(self,word):
        d=self.FindClosest(word)
        #print(d)
        if d.get('cls')=='':
            return np.array([0,0,0,0])
        else:
            #print(self.GetVal(word))
            return self.GetVal(d.get('cls'))
        return 0
    def RndVec(self):
        r = np.random.normal
        s=1
        v=np.array([r(0,s),r(0,s),r(0,s),r(0,s)])
        return v
    def GetRndVec(self,words,create=False,ret=True):#словарь строится за n!/k! операций, где n - кол-во слов. Переделать
        def GetOne(word,create=False,ret=True):
            d=self.FindClosest(word)
            p=np.exp(-d.get('dist')/5)
            #print(p)
            q=1-p
            v=self.RndVec()*q+self.FindVec(word)*p
            if (create == True) and (d.get('dist')!=0):
                self.Push(word, v)
            if ret:
                return v
        if ret:
            L=[]
        if type(words)==list:
            for elem in words:
                x=GetOne(elem,create,ret)
                if ret:
                    L.append(x)
        if type(words)==str:
            x=GetOne(words,create,ret)
            if ret:
                    L.append(x)
        if ret:
            return L

In [32]:
aim=getwords(words[0])
base=getwords(words[1])
contrast=getwords(words[2])
own=getwords(words[3])
stopwords=getwords(words[4])

In [39]:
ww=WordList()

ww.Push(aim,     np.array([1,0,0,0]))
ww.Push(base,    np.array([0,1,0,0]))
ww.Push(contrast,np.array([0,0,1,0]))
ww.Push(own,     np.array([0,0,0,1]))

ww.Push(stopwords,np.array([0,0,0,0]))

In [36]:
data_3=list()
for st in data_2:
    tmpstr1=st[0:4]
    tmpstr2=st[4:-1].strip()
    data_3.append([tmpstr1,tmpstr2])

In [20]:
import pandas as pd
import numpy as np

In [21]:
data_str=pd.DataFrame(data_3)

In [49]:
data_str[1][2]

'for mdl in general one can only have loss bounds which are finite but exponentially larger than those for bayes mixture'

In [43]:
for row in data_str[1]:
    ww.GetRndVec(row.split(' '),create=True,ret=False)

In [42]:
'ddd ddd'.split(' ')

['ddd', 'ddd']

In [44]:
len(ww.dict)

4453

In [45]:
ww.dict

{'we': array([0, 0, 0, 1]),
 'this': array([0, 0, 0, 1]),
 'paper': array([0, 0, 0, 1]),
 'study': array([0, 0, 0, 1]),
 'show': array([0, 0, 0, 1]),
 'present': array([0, 0, 0, 1]),
 'new': array([0, 0, 0, 1]),
 'model': array([0, 0, 0, 1]),
 'introduce': array([1, 0, 0, 0]),
 'current': array([1, 0, 0, 0]),
 'investigated': array([1, 0, 0, 0]),
 'compare': array([0, 0, 0, 1]),
 'designed': array([1, 0, 0, 0]),
 'argue': array([1, 0, 0, 0]),
 'examines': array([1, 0, 0, 0]),
 'propose': array([0, 0, 0, 1]),
 'interested': array([1, 0, 0, 0]),
 'address': array([1, 0, 0, 0]),
 'give': array([0, 0, 0, 1]),
 'discuss': array([0, 0, 0, 1]),
 'discover': array([1, 0, 0, 0]),
 'how': array([1, 0, 0, 0]),
 'identification': array([1, 0, 0, 0]),
 'developed': array([0, 0, 0, 1]),
 'influences': array([1, 0, 0, 0]),
 'main': array([1, 0, 0, 0]),
 'goal': array([1, 0, 0, 0]),
 'motivate': array([1, 0, 0, 0]),
 'establish': array([1, 0, 0, 0]),
 'applied': array([1, 0, 0, 0]),
 'formalize': arra

In [77]:
def createVecFromString(string,dictionary,size_t=30):
    arr=np.zeros(shape=(size_t,4))
    Li_words=string.split(' ')
    Li_words = list(filter(None, Li_words))
    #print(Li_words)
    arr_tmp=np.array(dictionary.GetRndVec(Li_words,create=False,ret=True))
    
    if len(arr_tmp)-len(arr)>0:
        print('Wrong size_t. Increase it')
    
    arr_start=np.random.randint(np.abs(len(arr_tmp)-len(arr)))
    print(len(arr_tmp))
    arr[arr_start:arr_start+len(arr_tmp)]=arr_tmp
    return arr
def createVecFromStringDefault(dictionary,size_t=30):
    def TmpFunc(string):
        return createVecFromString(string,dictionary,size_t)
    return TmpFunc

In [80]:
data_4=list()
classes={
    'misc':np.array([1,0,0,0,0]),
    'aimx':np.array([0,1,0,0,0]),
    'ownx':np.array([0,0,1,0,0]),
    'cont':np.array([0,0,0,1,0]),
    'base':np.array([0,0,0,0,1]),
}
F=createVecFromStringDefault(ww,size_t=100)
for row in data_3:
    label=row[0]
    para=row[1]
    data_4.append([classes.get(label),F(para)])

16
38
21
16
14
17
22
30
17
34
17
10
6
17
8
14
25
9
18
21
40
24
29
19
15
25
13
19
26
17
21
25
16
23
10
15
16
21
9
24
13
28
13
21
14
20
12
14
28
18
9
45
20
13
5
26
6
14
10
15
10
8
14
14
21
5
10
16
38
21
16
14
17
22
30
17
34
17
10
6
17
8
14
25
9
18
21
40
24
29
19
15
25
13
19
26
17
21
25
16
23
10
15
16
21
9
24
13
28
13
21
14
20
12
14
28
18
9
45
20
13
5
26
6
14
10
15
10
8
14
14
21
5
10
16
38
21
16
14
17
22
30
17
34
17
10
6
17
8
14
25
9
18
21
40
24
29
19
15
25
13
19
26
17
21
25
16
23
10
15
16
21
9
24
13
28
13
21
14
20
12
14
28
18
9
45
20
13
5
26
6
14
10
15
10
8
14
14
21
5
10
25
47
27
15
29
22
36
40
22
30
29
27
23
20
19
30
30
20
22
21
15
20
26
19
15
27
33
26
19
23
11
21
7
9
25
47
27
15
29
22
36
40
22
30
29
27
23
20
19
30
30
20
22
21
15
20
26
19
15
27
33
26
19
23
11
21
7
9
24
47
27
9
26
22
35
34
22
29
29
24
21
20
19
28
30
20
22
21
15
20
26
19
15
26
33
25
16
22
11
21
7
9
18
28
30
24
26
39
18
25
21
13
24
27
26
36
36
46
33
31
18
20
13
29
23
33
18
28
30
24
26
39
18
25
21
13
24
27
26
36
38
45
33
31

20
29
25
37
18
51
25
21
40
27
51
23
34
24
49
21
46
38
21
11
34
21
17
23
23
35
74
57
20
20
28
24
27
38
27
19
20
29
25
37
18
51
25
21
40
27
51
23
34
24
49
21
46
38
21
11
34
21
17
23
23
35
74
57
20
20
28
24
27
38
27
19
20
29
25
37
18
51
25
21
40
27
51
23
34
24
49
21
46
38
21
11
34
21
17
23
23
35
74
57
20
20
28
24
27
38
27
19
10
29
29
26
22
23
26
22
34
28
22
23
41
31
25
22
27
29
27
37
44
29
29
27
38
45
27
15
30
47
38
10
13
35
26
13
13
20
36
14
27
42
10
29
29
26
22
23
26
22
34
28
22
23
41
31
25
22
27
29
27
37
44
29
29
27
38
45
27
15
30
47
38
10
13
35
26
13
13
20
36
14
27
42
10
29
29
26
22
23
26
22
34
28
22
23
41
31
25
22
27
29
27
37
44
29
29
27
38
45
27
15
30
47
38
10
13
35
26
13
13
20
36
14
27
42
16
23
28
35
29
11
22
13
22
20
19
15
10
40
36
19
24
22
13
30
52
13
47
38
21
38
30
30
14
20
23
19
30
52
49
33
16
23
28
35
29
11
22
13
22
20
19
15
10
40
36
19
24
22
13
30
52
13
47
38
21
38
30
30
14
20
23
19
30
52
49
33
16
23
28
35
29
11
22
13
22
20
19
15
10
40
36
19
24
22
13
30
52
13
47
38
21
38
30
3

In [58]:
arr=np.zeros(shape=(30,4))

In [82]:
data_vec=pd.DataFrame(data_4)

In [83]:
data_vec.head()

,0,1
0,"[1, 0, 0, 0, 0]","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [..."
1,"[1, 0, 0, 0, 0]","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [..."
2,"[1, 0, 0, 0, 0]","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [..."
3,"[0, 1, 0, 0, 0]","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [..."
4,"[0, 0, 1, 0, 0]","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [..."


In [84]:
data_vec.to_csv('Vecoric.csv', sep='\t')

In [87]:
X=data_str[0]
y=data_str[1]

In [90]:
from sklearn.cross_validation import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [91]:
TRAIN_SIZE = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_SIZE, random_state=42)

In [92]:
import keras

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [95]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, X_func, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=5, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.X_func = X_func
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = self.X_func(X_train[ID])#применяем вышесозданную ф-ю

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [97]:
params = {'dim': (100,4),
          'batch_size': 64,
          'n_classes': 5,
          'n_channels': 1,
          'shuffle': True,
          'X_func': F
         }

In [98]:
from keras.models import Sequential

In [103]:
IDs_train=np.random.randint(len(X_train),size=1000)
IDs_test=np.random.randint(len(X_test),size=1000)

In [105]:
training_generator = DataGenerator(IDs_train, y_train, **params)
validation_generator  = DataGenerator(IDs_test, y_test, **params)

In [107]:

from keras.layers import Dense
from keras import regularizers

In [ ]:
model = Sequential()
model.add(Dense(40, input_shape=(100,4), activation='sigmoid'))
model.add(Dense(40, activation='sigmoid'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=1)